In [1]:
import os, sys
sys.path.append('../src/')

from copulae1 import *

import json
with open('../src/configs/config_CRIX_future_v3_MM.json','r') as file:
    config = json.load(file)
config
data_name = config['data_name']
calibration_method  = config['calibration_method']
OHR = pd.read_hdf('../results/'+data_name +'/'+calibration_method+'/best_h.h5')
if not os.path.exists("../results/" + data_name + "/MLE/reports"):
	print("Create new folder for results")
	os.mkdir("../results/" + data_name + "/MLE/reports")
	os.mkdir("../results/" + data_name + "/MM/reports")
    
if calibration_method == "MLE":
    report_path = "../results/" + data_name + "/MLE/reports/"
    result_path = "../results/" + data_name + "/MLE/"
elif calibration_method == "MM":
    report_path = "../results/" + data_name + "/MM/reports/"
    result_path = "../results/" + data_name + "/MM/"

In [2]:
OHR

0.csv                                                      \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05 VaR q=0.01 VaR q=0.05   
Gaussian         0.963379  0.933203  1.154492  0.959766   0.945312   0.867871   
t_Copula         0.931348  0.916699  1.334277  0.933398   0.850391   0.888867   
t_Copula_Capped  0.954199  0.927637  1.281738  0.956250   0.909375   0.880469   
Clayton          1.009766  0.896680  0.790527  0.860254   0.779687   0.927246   
Frank            0.896484  0.902246  1.017383  0.899609   0.907715   0.867871   
Gumbel           0.945996  1.009570  1.273340  1.058887   1.115039   0.965820   
Plackett         0.919141  0.915918  1.083594  0.921777   0.881250   0.893164   
Gauss Mix Indep  0.847168  0.912012  1.183887  0.729590   0.468750   0.895215   

                    1.csv                                ...     8.csv  \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05  ... ES q=0.01   
Gaussian         0.886816  0.872266  1.075684  0.894434  ...  0.983398   
t_Copula         0.850488  0.854297  1.203418  0.871289  ...  1.121484   
t_Copula_Capped  0.876465  0.865723  1.129004  0.886035  ...  1.046387   
Clayton          0.925488  0.841699  0.703125  0.792578  ...  0.610840   
Frank            0.828711  0.861230  0.957715  0.849805  ...  0.828027   
Gumbel           0.868262  0.949219  1.198730  1.002051  ...  1.147656   
Plackett         0.835645  0.855762  1.010059  0.848340  ...  0.854687   
Gauss Mix Indep  0.799512  0.848926  1.171289  0.794336  ...  1.053711   

                                                    9.csv                      \
                ES q=0.05 VaR q=0.01 VaR q=0.05  Variance  ERM k=10 ES q=0.01   
Gaussian         0.856543   0.817578   0.794141  0.839551  0.809277  0.933984   
t_Copula         0.806641   0.681055   0.795508  0.811523  0.794043  1.030469   
t_Copula_Capped  0.837793   0.770801   0.800391  0.825488  0.800195  0.987988   
Clayton          0.708301   0.628027   0.800781  0.874512  0.766895  0.608691   
Frank            0.802148   0.728516   0.830469  0.791504  0.796875  0.796191   
Gumbel           0.973828   0.983203   0.892383  0.825488  0.893555  1.106738   
Plackett         0.779395   0.722754   0.812695  0.784082  0.776855  0.815234   
Gauss Mix Indep  0.735156   0.487793   0.843750  0.763770  0.781250  0.931934   

                                                 
                ES q=0.05 VaR q=0.01 VaR q=0.05  
Gaussian         0.828223   0.793750   0.772266  
t_Copula         0.797070   0.744531   0.762695  
t_Copula_Capped  0.811230   0.759766   0.779980  
Clayton          0.710840   0.628613   0.816016  
Frank            0.765918   0.703125   0.802344  
Gumbel           0.937109   0.935352   0.838867  
Plackett         0.745605   0.705859   0.784277  
Gauss Mix Indep  0.723926   0.499902   0.809082  

[8 rows x 438 columns]

In [370]:
natp_path = '../_mathematica/data/'
natp_ls = os.listdir(natp_path)
natp_h = [l for l in natp_ls if l.endswith('_h.csv')]
file_names = [l.replace('_h','') for l in natp_h]
OHR_combined = []
for i in range(len(natp_h)):
    _natp_h = pd.DataFrame(open(natp_path+natp_h[i], 'r').readlines())
    _natp_h = _natp_h.iloc[1:7,:]
    file_name = natp_h[i].replace('_h','')
    for i in range(len(_natp_h)):
        _natp_h.iloc[i,0] = np.float32(_natp_h.iloc[i,0].replace('\n',''))
    _natp_h.columns = ['NIG_factor']
    _natp_h.index = ['Variance', 'VaR q=0.01', 'VaR q=0.05', 'ES q=0.01', 'ES q=0.05', 'ERM k=10']
    _natp_h = _natp_h.T
    
    OHR_combined.append(OHR.loc[:,file_name].append(_natp_h))
OHR = pd.concat(OHR_combined, axis=1, keys=file_names).dropna(axis=1)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep', 'NIG_factor']
Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']

OHR = OHR.loc[Copula_names,:]

In [4]:
temp = []
for C in Copula_names:
    hs_by_Copula = OHR.loc[C:C,:].melt()
    hs_by_Copula.columns = ['file_name', 'risk_measure', 'OHR']
    hs_by_Copula.loc[:, 'copula'] = C
    temp.append(hs_by_Copula)

OHR_to_math = pd.concat(temp, axis=0)

In [5]:
v5_path = '../processed_data/'+config['data_name'].replace('3', '5')+'/test/'
v3_path = '../processed_data/'+config['data_name']+'/test/'

v5_files = os.listdir(v5_path)
v3_files = os.listdir(v3_path)

In [6]:
pd.read_csv(v5_path+'0.csv').tail()

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
95,95,2020-04-28,18379.326121,7785.0,0.002615,-0.001284
96,96,2020-04-27,18331.330183,7795.0,0.037923,0.024019
97,97,2020-04-24,17649.164021,7610.0,0.012686,-0.006549
98,98,2020-04-23,17426.675218,7660.0,0.035459,0.067502
99,99,2020-04-22,16819.578224,7160.0,0.040012,0.035540


In [7]:
pd.read_csv(v3_path+'19.csv') 

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
0,95,2020-04-28,18379.326121,7785.0,0.002615,-0.001284
1,96,2020-04-27,18331.330183,7795.0,0.037923,0.024019
2,97,2020-04-24,17649.164021,7610.0,0.012686,-0.006549
3,98,2020-04-23,17426.675218,7660.0,0.035459,0.067502
4,99,2020-04-22,16819.578224,7160.0,0.040012,0.035540


In [8]:
OHR_to_use = OHR_to_math.copy()
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: int(x.replace('.csv',''))-19) # shift file names
OHR_to_use = OHR_to_use.loc[OHR_to_use.file_name >0,:] # remove redudant rows
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: str(x)+'.csv')

In [9]:
risk_measure = list(OHR_to_use.risk_measure.unique())
ls = list(OHR_to_use.file_name.unique())

In [10]:
OHR_to_use.loc[:,"HE"] = 0 # Add a new column for results of HE

In [11]:
OHR_to_use.head()

,file_name,risk_measure,OHR,copula,HE
78,1.csv,Variance,0.860937,Gaussian,0
79,1.csv,ERM k=10,0.814258,Gaussian,0
80,1.csv,ES q=0.01,0.989648,Gaussian,0
81,1.csv,ES q=0.05,0.835352,Gaussian,0
82,1.csv,VaR q=0.01,0.788379,Gaussian,0


In [12]:
def hedging_effectiveness2(rm, rs, rf, h):
    rh = rs - h*rf
    if rm == 'ERM k=10':
        return(1-ERM_estimate_trapezoidal(10, rh)/ERM_estimate_trapezoidal(10, rs))

    elif rm == 'ES q=0.01':
        return(1-ES(0.01, rh)/ES(0.01, rs))

    elif rm == 'ES q=0.05':
        return(1-ES(0.05, rh)/ES(0.05, rs))

    elif rm == 'VaR q=0.01':
        return(1-VaR(0.01, rh)/VaR(0.01, rs))

    elif rm == 'VaR q=0.05':
        return(1-VaR(0.05, rh)/VaR(0.05, rs))

    elif rm == 'Variance':
        return(1-Variance(rh)/Variance(rs))

In [13]:
def rename_rm(rm):
    rm = rm.replace('q=','')
    rm = rm.replace('0.01', '99%')
    rm = rm.replace('0.05', '95%')
    return rm

In [15]:
config['spot_name']

'log return CRIX'

In [16]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,-1] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)

In [17]:
i_id = OHR_to_use.risk_measure == rm
j_id = OHR_to_use.copula == C

In [18]:
path = '../latex/_pics/CRIX_future_hedging_effectiveness_100days_with_v3_OHR/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(10,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)), Copula_names)
    plt.savefig(path+'%s_100days_with_v3_OHR.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()

In [19]:
import itertools
a = ['HR 1']
b = risk_measure.copy()
c = ls.copy()
d = [1]
naive = pd.DataFrame(list(itertools.product(*[a,b,c,d])))
naive.columns = ['copula', 'risk_measure', 'file_name', 'OHR']
naive.loc[:,'HE'] = 0

In [20]:
OHR_to_use = OHR_to_use.append(naive)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,0] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)


In [22]:
path = '../latex/_pics/CRIX_future_hedging_effectiveness_100days_with_v3_OHR_with_naive/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(12,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
   
    j_id = OHR_to_use.copula == 'HR 1'
    plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j+1])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)+1), Copula_names+['HR 1'])
    plt.savefig(path+'%s_100days_with_v3_OHR_with_naive.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()
